In [1]:
import pickle
import dimod

hubo = None
with open('hubo.pkl', 'rb') as f:
    hubo = pickle.load(f)
print(len(hubo))
hubo, offset = hubo["hubo"], hubo["offset"]
print(len(hubo))
new_hubo = {}

i = 0
for k, v in hubo.items():
    new_hubo[k] = int(v)

hubo = new_hubo
binary_polynomial = dimod.BinaryPolynomial.from_hubo(hubo, offset)
binary_polynomial.normalize()
hubo, offset = binary_polynomial.to_hubo()

2
98112


In [2]:
from utils import solve_bqm_in_leap


bqm = dimod.make_quadratic(hubo, 2, dimod.BINARY)
bqm.offset = offset
bqm.normalize()
print("Offset: ", bqm.offset)
print("Linear: ", len(bqm.linear))
print("Quadratic: ", len(bqm.quadratic))

sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")

Offset:  0.16666666666666666
Linear:  6855
Quadratic:  114393


In [3]:
import cplex
import time


def qubo_to_lp(bqm, identifier):
    cplex_problem = cplex.Cplex()
    cplex_problem.objective.set_sense(cplex_problem.objective.sense.minimize)
    variable_symbols = [str(var) for var in bqm.variables]
    cplex_problem.variables.add(names=variable_symbols, 
                                        types=[cplex_problem.variables.type.binary]*len(variable_symbols))

    linear_coeffs = bqm.linear
    obj_list = [(str(name), coeff) for name, coeff in linear_coeffs.items()]
    cplex_problem.objective.set_linear(obj_list)

    quadratic_coeffs = bqm.quadratic
    obj_list = [(str(name[0]), str(name[1]), coeff) for name, coeff in quadratic_coeffs.items()]
    cplex_problem.objective.set_quadratic_coefficients(obj_list)
    lp_file = "linear_program_files//" + str(identifier) + ".lp"
    cplex_problem.write(lp_file)
    return cplex_problem


def solve_with_cplex_solver(cplex_problem, offset, print_log = False):
    samplesets = {}
    if not print_log:
        cplex_problem.set_log_stream(None)
        cplex_problem.set_error_stream(None)
        cplex_problem.set_warning_stream(None)
        cplex_problem.set_results_stream(None)
    time_start = time.time()
    cplex_problem.solve()
    time_end = time.time()
    elapsed_time = time_end - time_start
    status = cplex_problem.solution.get_status_string()
    result = cplex_problem.solution.get_values()
    variables = cplex_problem.variables.get_names()
    value = cplex_problem.solution.get_objective_value() + offset
    samplesets["cplex"] = { "status": status, "energy": value, 
                                "time": elapsed_time, "result": dict(zip(variables, result)) }
    return samplesets

In [4]:
cplex_problem = qubo_to_lp(bqm, "matrix_mult")
samplesets = solve_with_cplex_solver(cplex_problem, bqm.offset)

Default row names c1, c2 ... being created.


In [3]:
import numpy as np
from itertools import combinations
from utils import solve_bqm_in_leap

def flatten(data):
    if isinstance(data, tuple):
        for x in data:
            yield from flatten(x)
    else:
        yield str(data)

def pair_to_index(x, dim):
    return dim*(x[0] - 1) + x[1] - 1

def get_standard_tensor(dim):
    initial_tensor = np.zeros((dim**2, dim**2, dim**2))
    for c1 in range(1, dim + 1):
        for c2 in range(1, dim + 1):
            c = (c1, c2)
            for a2 in range(1, dim + 1):
                a = (c1, a2)
                b = (a2, c2)
                initial_tensor[pair_to_index(a, dim)][pair_to_index(b, dim)][pair_to_index(c, dim)] = 1
    return initial_tensor

def square_negative_sum(hubo, variables, offset):
    for v in variables:
        #v = tuple(sorted(list((flatten(var)))))
        if offset == 1:
            if v in hubo:
                hubo[v] -= 1
            else:
                hubo[v] = -1
        elif offset == 0:
            if v in hubo:
                hubo[v] += 1
            else:
                hubo[v] = 1
    combs = combinations(variables, 2)
    for pair in combs:
        v = tuple(sorted(list((flatten(pair)))))
        if v in hubo:
            hubo[v] += 2
        else:
            hubo[v] = 2
    return hubo

def construct_all_tensors(sample, dim, suggested_optimal):
    print(sample)
    positive_linear_vars = []
    tensors = []
    for i in range(suggested_optimal):
        x, y, z = [], [], []
        for j in range(dim**2):
            x.append(sample["l_" + str(i) + "x" + str(j)] - sample["r_" + str(i) + "x" + str(j)])
            y.append(sample["l_" + str(i) + "y" + str(j)] - sample["r_" + str(i) + "y" + str(j)])
            z.append(sample["l_" + str(i) + "z" + str(j)] - sample["r_" + str(i) + "z" + str(j)])
        tensors.append([x,y,z])
    return tensors

In [4]:
tensors = construct_all_tensors(sample, 2, 7)

initial_tensor = get_standard_tensor(2)
for t in tensors:
    print(t)
    initial_tensor = np.mod(initial_tensor - np.tensordot(t[0], np.tensordot(t[1], t[2], axes=0), axes=0), 2)

# After substraction, the tensor should be the origo tensor (all entries are 0)
if np.count_nonzero(initial_tensor.flatten()) == 0:
    print("Success!")
else:
    print("No success")
    
print(initial_tensor)

{'l_0x0': 0, 'l_0x0*l_0y0': 1, 'l_0x0*l_0y0*l_0z0': 1, 'l_0x0*l_0y0*l_0z2': 1, 'l_0x0*l_0y0*l_0z3': 1, 'l_0x0*l_0y0*l_3x0*l_3y0': 1, 'l_0x0*l_0y0*r_0z0': 0, 'l_0x0*l_0y0*r_0z1': 1, 'l_0x0*l_0y0*r_0z2': 0, 'l_0x0*l_0y0*r_0z3': 0, 'l_0x0*l_0y0*r_2y0*l_2x0': 1, 'l_0x0*l_0y0*r_3x0*l_3y0': 1, 'l_0x0*l_0y2*l_0z0': 0, 'l_0x0*l_0y2*l_0z2': 0, 'l_0x0*l_0y2*l_0z3': 0, 'l_0x0*l_0y2*r_0z0': 0, 'l_0x0*l_0y2*r_0z1': 0, 'l_0x0*l_0y2*r_0z2': 0, 'l_0x0*l_0y2*r_0z3': 0, 'l_0x0*l_0y3*r_0z2': 0, 'l_0x0*l_0z0': 0, 'l_0x0*l_0z0*l_0y3': 0, 'l_0x0*l_0z1': 0, 'l_0x0*l_0z1*l_0y2': 0, 'l_0x0*l_0z1*l_0y3': 0, 'l_0x0*l_0z1*l_0y3*r_0z1': 0, 'l_0x0*l_0z1*r_0y1': 0, 'l_0x0*l_0z1*r_0z1': 0, 'l_0x0*l_0z2*l_0y3': 0, 'l_0x0*l_0z3*l_0y3': 0, 'l_0x0*l_6x0': 0, 'l_0x0*r_0y0': 0, 'l_0x0*r_0y0*l_0z0': 0, 'l_0x0*r_0y0*l_0z1*r_0z1': 0, 'l_0x0*r_0y0*l_0z2': 0, 'l_0x0*r_0y0*l_0z3': 0, 'l_0x0*r_0y0*r_0z0': 0, 'l_0x0*r_0y0*r_0z1': 0, 'l_0x0*r_0y0*r_0z2': 0, 'l_0x0*r_0y0*r_0z3': 0, 'l_0x0*r_0y0*r_2y0*l_2x0': 0, 'l_0x0*r_0y0*r_3x0*l_

In [5]:
strassen_tensors = [[[0,0,0,1], [-1,0,1,0], [1,0,1,0]],
                        [[1,1,0,0], [0,0,0,1], [-1,1,0,0]],
                        [[-1,0,1,0], [1,1,0,0], [0,0,0,1]],
                        [[1,0,0,1], [1,0,0,1], [1,0,0,1]],
                        [[0,1,0,-1], [0,0,1,1], [1,0,0,0]],
                        [[1,0,0,0], [0,1,0,-1], [0,1,0,1]],
                        [[0,0,1,1], [1,0,0,0], [0,0,1,-1]]]

variables_to_values = dict()
for i in range(7):
    for j in range(2**2):
        if strassen_tensors[i][0][j] == 1:
            variables_to_values["l_" + str(i) + "x" + str(j)] = 1
            variables_to_values["r_" + str(i) + "x" + str(j)] = 0
        elif strassen_tensors[i][0][j] == -1:
            variables_to_values["l_" + str(i) + "x" + str(j)] = 0
            variables_to_values["r_" + str(i) + "x" + str(j)] = 1
        else:
            variables_to_values["l_" + str(i) + "x" + str(j)] = 0
            variables_to_values["r_" + str(i) + "x" + str(j)] = 0
        
        if strassen_tensors[i][1][j] == 1:
            variables_to_values["l_" + str(i) + "y" + str(j)] = 1
            variables_to_values["r_" + str(i) + "y" + str(j)] = 0
        elif strassen_tensors[i][1][j] == -1:
            variables_to_values["l_" + str(i) + "y" + str(j)] = 0
            variables_to_values["r_" + str(i) + "y" + str(j)] = 1
        else:
            variables_to_values["l_" + str(i) + "y" + str(j)] = 0
            variables_to_values["r_" + str(i) + "y" + str(j)] = 0
                
        if strassen_tensors[i][2][j] == 1:
            variables_to_values["l_" + str(i) + "z" + str(j)] = 1
            variables_to_values["r_" + str(i) + "z" + str(j)] = 0
        elif strassen_tensors[i][2][j] == -1:
            variables_to_values["l_" + str(i) + "z" + str(j)] = 0
            variables_to_values["r_" + str(i) + "z" + str(j)] = 1
        else:
            variables_to_values["l_" + str(i) + "z" + str(j)] = 0
            variables_to_values["r_" + str(i) + "z" + str(j)] = 0
                    
                    
for v in bqm.variables:
    if v not in variables_to_values:
        elems = v.split("*")
        multiplication_result = 1
        for e in elems:
           multiplication_result = multiplication_result * variables_to_values[e]
        variables_to_values[v] = multiplication_result
energy_bqm = bqm.energy(variables_to_values)

binary_polynomial = dimod.BinaryPolynomial.from_hubo(hubo, 8.0)
energy = binary_polynomial.energy(variables_to_values)
print("BQM: ", energy_bqm)
print("Binary Polynomial: ", energy)


BQM:  2.9976021664879227e-14
Binary Polynomial:  7.0
